In [ ]:
! pip install numpy
! pip install openai
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos
! pip install tenacity
! pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004

In [1]:
import json
import datetime
import time

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
from azure.cosmos import exceptions, CosmosClient, PartitionKey
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.models import Vector
from azure.search.documents.indexes.models import (
    IndexingSchedule,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchField,
    SearchFieldDataType,
    SearchableField,
    SemanticConfiguration,
    SimpleField,
    PrioritizedFields,
    SemanticField,
    SemanticSettings,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    SearchIndexerDataSourceConnection
)

import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [2]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "rag.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

cosmosdb_endpoint = config['cosmos_db_api_endpoint']
cosmosdb_key = config['cosmos_db_api_key']
cosmosdb_connection_str = config['cosmos_db_connection_string']

cog_search_endpoint = config['cognitive_search_api_endpoint']
cog_search_key = config['cognitive_search_api_key']

openai.api_type = config['openai_api_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']
embeddings_deployment = config['openai_embeddings_deployment']
completions_deployment = config['openai_completions_deployment']

In [3]:
# from datasets import load_dataset
# import pandas as pd 

# dataset = load_dataset("squad_v2")
# df = pd.DataFrame.from_dict(dataset["train"])

# samurai_df = df[df["title"] == "Samurai"]

# #Our sentences we like to encode
# sentences = samurai_df.question.tolist()

# #Sentences are encoded by calling model.encode()
# samurai_questions_embedded = model.encode(sentences)

# data = {
#     "embeddings": samurai_questions_embedded, # this is a list
#     "payload": sentences, # this is also a list
#     "@search.action": ["upload"] * len(samurai_questions_embedded)
# }

c:\Users\leopu\OneDrive\Programming\Python\azure\azure-rag\azure-rag-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Load text-sample.json data file
data_file = open(file="text-sample.json", mode="r")
#data_file = open(file="../../DataSet/AzureServices/text-sample_w_embeddings.json", mode="r") # load this file instead if embeddings were previously created and saved.
data = json.load(data_file)
data_file.close()

In [24]:
# Take a peek at one data item
print(data[0])

{'id': '1', 'title': 'Azure App Service', 'content': 'Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.', 'category': 'Web'}


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("intfloat/e5-small-v2")

def generate_embeddings(text):
    return model.encode(text)

c:\Users\leopu\OneDrive\Programming\Python\azure\azure-rag\azure-rag-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# Generate embeddings for title and content fields
for item in data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings.tolist()
    item['contentVector'] = content_embeddings.tolist()
    item['@search.action'] = 'upload'

# Save embeddings to sample_text_w_embeddings.json file
with open("text-sample_w_embeddings.json", "w") as f:
    json.dump(data, f)

## Upload data to Cosmos DB

In [10]:
# Create the client to interact with the Azure Cosmos DB resource
client = CosmosClient(cosmosdb_endpoint, cosmosdb_key)

In [11]:
# Create a database in Azure Cosmos DB.
try:
    database = client.create_database_if_not_exists(id="cosmosvectorstore")
    print(f"Database created: {database.id}")

except exceptions.CosmosResourceExistsError:
    print("Database already exists.")

Database created: cosmosvectorstore


In [12]:
# Create a container in Azure Cosmos DB.
try:
    partition_key_path = PartitionKey(path="/id")
    container = database.create_container_if_not_exists(
        id="AzureServices",
        partition_key=partition_key_path,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: AzureServices


In [36]:
# Create data items for every entry in the dataset, insert them into the database and collection specified above.
for data_item in data:
    try:
        container.create_item(body=data_item)
    
    except exceptions.CosmosResourceExistsError:
        print("Data item already exists.")

## Create search index in CognitiveSearch

In [3]:
# Create index
cog_search_cred = AzureKeyCredential(cog_search_key)
index_name = "cosmosdb-vector-search-index"

# Create a search index and define the schema (names, types, and parameters)
index_client = SearchIndexClient(
    endpoint=cog_search_endpoint, credential=cog_search_cred)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=384, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=384, vector_search_configuration="my-vector-config"),
]

# Configure vector search
vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

# Configure semantic search. This will allow us to conduct semantic or hybrid search (with vector search) later on if desired.
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f'{result.name} created')

cosmosdb-vector-search-index created


In [5]:
# create an indexer to pull data from CosmosDB into Cognitive Search 
def _create_datasource():
    # Here we create a datasource. 
    ds_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
    container = SearchIndexerDataContainer(name="AzureServices")
    data_source_connection = SearchIndexerDataSourceConnection(
        name="cosmosdb-tutorial-indexer", type="cosmosdb", connection_string=cosmosdb_connection_str, container=container
    )
    data_source = ds_client.create_or_update_data_source_connection(data_source_connection)
    return data_source

ds_name = _create_datasource().name

indexer = SearchIndexer(
        name="cosmosdb-tutorial-indexer",
        data_source_name=ds_name,
        target_index_name=index_name)

indexer_client = SearchIndexerClient(cog_search_endpoint, cog_search_cred)
indexer_client.create_or_update_indexer(indexer)  # create the indexer

result = indexer_client.get_indexer("cosmosdb-tutorial-indexer")
print(result)

# Run the indexer we just created.
indexer_client.run_indexer(result.name)

{'additional_properties': {'@odata.context': 'https://cognitivevectorsearch.search.windows.net/$metadata#indexers/$entity'}, 'name': 'cosmosdb-tutorial-indexer', 'description': None, 'data_source_name': 'cosmosdb-tutorial-indexer', 'skillset_name': None, 'target_index_name': 'cosmosdb-vector-search-index', 'schedule': None, 'parameters': None, 'field_mappings': [], 'output_field_mappings': [], 'is_disabled': False, 'e_tag': '"0x8DBE934CD170578"', 'encryption_key': None, 'cache': None}


In [7]:
# simple function to assist with vector search
def vector_search(query):
    search_client = SearchClient(cog_search_endpoint, index_name, cog_search_cred)
    results = search_client.search(
        search_text="",
        vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),
        select=["title", "content", "category"]
    )
    return results

In [11]:
query = "serverless tools for software development"  
results = vector_search(query)
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  

Title: Azure Functions
Score: 0.8505801
Content: Azure Functions is a serverless compute service that enables you to run code on-demand without having to manage infrastructure. It allows you to build and deploy event-driven applications that automatically scale with your workload. Functions support various languages, including C#, F#, Node.js, Python, and Java. It offers a variety of triggers and bindings to integrate with other Azure services and external services. You only pay for the compute time you consume.
Category: Compute

Title: Azure Web PubSub
Score: 0.8489944
Content: Azure Web PubSub is a fully managed, real-time messaging service that enables you to build and scale real-time web applications using WebSockets. It provides features like automatic scaling, custom domains, and serverless integration. Web PubSub supports various programming languages, such as C#, JavaScript, and Java. You can use Azure Web PubSub to build chat applications, real-time dashboards, and collaborat